# Firebase in python

Bestudeer de API van Firebase die je kan vinden op [deze link](https://firebase.google.com/docs/firestore/quickstart).
Schrijf de nodige code in de bijhorende onderstaande cellen om de volgende stappen uit te voeren (voer de queries uit in de firestore):

* Maak een connectie met firebase vanuit python
* Upload alle documenten het restaurants.json bestand naar een collectie op firebase. Zorg ervoor dat de collectie "restaurants" noemt en gebruik het id van het restaurant als document identifier. Let er hierbij op dat elke lijn 1 document voor stelt waardoor je dus dit bestand lijn per lijn moet inlezen. Bekijk de data in de cloud firestore console via de website. Kan je de data zien die toegevoegd is?
* Selecteer het document in de firestore met restaurant id "40356731"
* Hoeveel bakkerijken zijn er aanwezig in de datastore?
* Zoek alle restauranten met zipcode "11234" op
* Wat zijn de 5 laatste (op basis van alfabet) restaurants met een "American" of "Italian" keuken

In [1]:
# connecteer met firebase
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore

# Use a service account.
cred = credentials.Certificate('../../bigdata_firebase.json')

app = firebase_admin.initialize_app(cred)

db = firestore.client()

In [2]:
# upload alle documenten uit het restaurants.json bestand
import json 

coll_restaurants = db.collection("restaurants")

with open("restaurants.json") as f:
    lines = f.readlines()
    
    for line in lines:
        line = json.loads(line)
        coll_restaurants.document(line["restaurant_id"]).set(line)

In [5]:
# Selecteer het document in de firestore met restaurant id "40356731"
print(coll_restaurants.document("40356731").get().to_dict())

{'grades': [{'grade': 'A', 'date': {'$date': 1405296000000}, 'score': 12}, {'grade': 'A', 'score': 8, 'date': {'$date': 1373414400000}}, {'score': 5, 'date': {'$date': 1341964800000}, 'grade': 'A'}, {'grade': 'A', 'date': {'$date': 1329955200000}, 'score': 8}], 'address': {'building': '1839', 'coord': [-73.9482609, 40.6408271], 'street': 'Nostrand Avenue', 'zipcode': '11226'}, 'name': 'Taste The Tropics Ice Cream', 'borough': 'Brooklyn', 'cuisine': 'Ice Cream, Gelato, Yogurt, Ices', 'restaurant_id': '40356731'}


In [7]:
# Hoeveel bakkerijen zijn er aanwezig in de datastore?

docs = coll_restaurants.where(u'cuisine', u'==', 'Bakery').get()

print("Aantal bakkerijen:", len(docs))

Aantal bakkerijen: 127


In [9]:
# zoek alle restauranten met zipcode "11234" op

docs = coll_restaurants.where(u'address.zipcode', u'==', '11234').stream()

for doc in docs:
    print(f'{doc.id} => {doc.to_dict()}')

40356483 => {'restaurant_id': '40356483', 'cuisine': 'Delicatessen', 'address': {'street': 'Avenue U', 'building': '7114', 'coord': [-73.9068506, 40.6199034], 'zipcode': '11234'}, 'borough': 'Brooklyn', 'name': "Wilken'S Fine Food", 'grades': [{'score': 10, 'date': {'$date': 1401321600000}, 'grade': 'A'}, {'grade': 'A', 'score': 10, 'date': {'$date': 1389657600000}}, {'date': {'$date': 1375488000000}, 'grade': 'A', 'score': 8}, {'date': {'$date': 1342569600000}, 'grade': 'A', 'score': 10}, {'date': {'$date': 1331251200000}, 'grade': 'A', 'score': 13}, {'grade': 'A', 'score': 9, 'date': {'$date': 1318550400000}}]}
40367164 => {'grades': [{'date': {'$date': 1392854400000}, 'score': 10, 'grade': 'A'}, {'date': {'$date': 1372636800000}, 'score': 15, 'grade': 'B'}, {'date': {'$date': 1357776000000}, 'score': 15, 'grade': 'B'}, {'date': {'$date': 1343001600000}, 'score': 11, 'grade': 'A'}, {'score': 7, 'grade': 'A', 'date': {'$date': 1325635200000}}, {'grade': 'C', 'date': {'$date': 13163904

In [10]:
# Wat zijn de 5 laatste (op basis van alfabet) restaurants met een "American" of "Italian" keuken
coll_restaurants.order_by(u'name', direction=firestore.Query.DESCENDING).limit(5)